In [2]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from polygon import RESTClient
from polygon.rest.models.indicators import (
    SMAIndicatorResults,
    EMAIndicatorResults,
    RSIIndicatorResults,
    MACDIndicatorResults,
)

In [3]:
df = pd.read_csv('snp100.csv', header=None)
snp100tickers = df[0]

In [4]:
key = os.environ.get('API_KEY')

In [5]:
client = RESTClient(key)

def download_data(from_date, to_date, ticker, key):
    '''Downloads trading data for each ticker for the giving
    period in format Y-M-D and returns the Dataframe in the csv format'''
    client = RESTClient(f"{key}")
    aggs = []
    for a in client.list_aggs(
        f"{ticker}",
        1,
        "day",
        f"{from_date}",
        f"{to_date}",
        limit=5000,
        ):
        aggs.append(a)
# creating the basic dataframe
    data = pd.DataFrame(aggs)
# adding the columns with new features
# 25 days simple moving average
    sma25 = client.get_sma(
        f"{ticker}",
        timestamp_gte=f"{from_date}",
        timestamp_lte=f"{to_date}",
        window=25,
        series_type='close',
        limit=5000,
        )
    data_sma25 = pd.DataFrame(sma25.values)
    data = pd.merge(data, data_sma25, on='timestamp', how='left').rename(columns={'value': 'sma25'})
# 100 days simple moving average    
    sma100 = client.get_sma(
        f"{ticker}",
        timestamp_gte=f"{from_date}",
        timestamp_lte=f"{to_date}",
        window=100,
        series_type='close',
        limit=5000,
        )
    data_sma100 = pd.DataFrame(sma100.values)
    data = pd.merge(data, data_sma100, on='timestamp', how='left').rename(columns={'value':'sma100'})
 # 200 days simple moving average    
    sma200 = client.get_sma(
        f"{ticker}",
        timestamp_gte=f"{from_date}",
        timestamp_lte=f"{to_date}",
        window=200,
        series_type='close',
        limit=5000,
        )
    data_sma200 = pd.DataFrame(sma200.values)    
    data = pd.merge(data, data_sma200, on='timestamp', how='left').rename(columns={'value':'sma200'})
 # 14 days relative strength index    
    rsi = client.get_rsi(
        f"{ticker}",
        timestamp_gte=f"{from_date}",
        timestamp_lte=f"{to_date}",
        window=14,
        series_type='close',
        limit=5000,
        )
    data_rsi = pd.DataFrame(rsi.values)    
    data = pd.merge(data, data_rsi, on='timestamp', how='left').rename(columns={'value': 'rsi'})
# moving average convergence/divergence
    macd = client.get_macd(
        f"{ticker}",
        timestamp_gte=f"{from_date}",
        timestamp_lte=f"{to_date}",
        short_window=12,
        long_window=26,
        signal_window=9,
        series_type='close',
        limit=5000,
        )
    data_macd = pd.DataFrame(macd.values)    
    data = pd.merge(data, data_macd, on='timestamp', how='left').rename(columns={'value':'macd'})
# adding volatility column
    # avg_price = np.sum(data['close'])/len(data['close'])
    # data['volatility'] = np.sqrt(np.sum(data['close']-avg_price**2)/(len(data['close'])-1))   
    return data.to_csv(f'raw_data/{ticker}.csv')

In [6]:
for ticker in snp100tickers:
    download_data('2018-08-30', '2023-08-30', ticker, key=key)